In [1]:
function xₙ(n)
	if n == 0
		return x₀
	elseif n == 1
		return x₁
	else
		return (3*xₙ(n-1) + 2*xₙ(n-2)) % 19 + 2
	end
end

xₙ (generic function with 1 method)

In [2]:
ID = 6330401742
x₀ = ID % 100
x₁ = 7
x₂ = xₙ(2)
ω̂ = 5^x₂ % 23
println("x₂ = ", x₂)
println("ω̂ = ", ω̂)

x₂ = 12
ω̂ = 18


In [3]:
# ultilities function
function construct_asset_binomial_tree(S, u, v, n, m)
    asset = zeros(n, m)
    for i in 1:n
        for j in 1:i
            asset[j, i] = S * u^(i-j) * v^(j-1)
        end
    end
    return asset
end

# Function to calculate the option payoff for a call option
function call_option_payoff(S, E)
    return max(S - E, 0.0)
end

# Function to compute the option tree
function compute_option_tree(asset_tree, E, r, δt, p′)
    n, m = size(asset_tree)
    V = zeros(n, m)

    discount_factor = 1 / (1 + r * δt)

    # Populate the option values at maturity
    for i in 1:n
        V[i, n] = call_option_payoff(asset_tree[i, n], E)
    end

    # Backward induction to fill in the rest of the option tree
    for i in (n-1):-1:1
        for j in 1:i
            V⁺ = V[j, i+1]
            V⁻ = V[j+1, i+1]
            V[j, i] = discount_factor * (p′ * V⁺ + (1 - p′) * V⁻)
        end
    end

    return V
end

compute_option_tree (generic function with 1 method)

In [4]:
S = 101.0
E = 101.0
r = 10.0 / 100.0
σ = 0.2 + ω̂ / 100 # already given this value
δt = 1 / 12
# derived u, v, p₁ from the given formula
u = 1 + σ * sqrt(δt)
v = 1 - σ * sqrt(δt)
p′ = 1/2 + r*sqrt(δt)/2/σ
println("σ = ", σ)
println("u = ", u)
println("v = ", v)
println("p′ = ", p′)

σ = 0.38


u = 1.1096965511460288
v = 0.8903034488539712
p′ = 0.5379835703414227


In [5]:
# construct the asset tree
asset = construct_asset_binomial_tree(S, u, v, 5, 5)

5×5 Matrix{Float64}:
 101.0  112.079   124.374   138.017   153.158
   0.0   89.9206   99.7846  110.731   122.877
   0.0    0.0      80.0567   88.8386   98.5839
   0.0    0.0       0.0      71.2747   79.0933
   0.0    0.0       0.0       0.0      63.4561

In [6]:
# compute option using asset tree
option = compute_option_tree(asset, E, r, δt, p′)

5×5 Matrix{Float64}:
 10.3163  16.4821  25.5438   37.8522  52.1575
  0.0      3.3227   6.22768  11.6724  21.8774
  0.0      0.0      0.0       0.0      0.0
  0.0      0.0      0.0       0.0      0.0
  0.0      0.0      0.0       0.0      0.0

In [7]:
# change the volatility parameter
σ = 0.25 + ω̂ / 100
u = 1 + σ * sqrt(δt)
v = 1 - σ * sqrt(δt)
p′ = 1/2 + r*sqrt(δt)/2/σ
println("σ = ", σ)
println("u = ", u)
println("v = ", v)
println("p′ = ", p′)

σ = 0.43
u = 1.1241303078757696
v = 0.8758696921242305
p′ = 0.5335668761156759


In [8]:
# construct new asset tree 
new_asset = construct_asset_binomial_tree(S, u, v, 5, 5)

5×5 Matrix{Float64}:
 101.0  113.537   127.631   143.473   161.283
   0.0   88.4628   99.4438  111.788   125.664
   0.0    0.0      77.4819   87.0998   97.9115
   0.0    0.0       0.0      67.8641   76.2881
   0.0    0.0       0.0       0.0      59.4401

In [9]:
# compute option using new asset tree
new_option = compute_option_tree(new_asset, E, r, δt, p′)

5×5 Matrix{Float64}:
 11.4882  18.5158   28.954   43.3081  60.2828
  0.0      3.65441   6.9061  13.0511  24.664
  0.0      0.0       0.0      0.0      0.0
  0.0      0.0       0.0      0.0      0.0
  0.0      0.0       0.0      0.0      0.0